In [1]:
import pandas as pd
import numpy as np

pd.set_option('display.max_columns', None)

print("Libraries loaded successfully.")

Libraries loaded successfully.


In [3]:
file_path = "C:/Users/91901/Downloads/synthetic_fb_ads_undergarments.csv" 
df = pd.read_csv(file_path)

print(f"Data loaded from: {file_path}")

Data loaded from: C:/Users/91901/Downloads/synthetic_fb_ads_undergarments.csv


In [5]:
print("--- First 5 Rows (Head) ---")
print(df.head())

print("\n--- Column Information (Data Types and Missing Values) ---")
df.info()

--- First 5 Rows (Head) ---
             campaign_name        adset_name        date   spend  impressions  \
0    Men ComfortMax Launch  Adset-1 Retarget  2025-01-01  640.09       235597   
1    Men ComfortMax Launch  Adset-1 Retarget  2025-01-02  373.75       276194   
2  Men  ComfortMax  Launch  Adset-1 Retarget  2025-01-03  703.79       466572   
3    Men_ComfortMax_Launch  Adset-1 Retarget  2025-01-04  441.50       193230   
4  Men  ComfortMax  Launch  Adset-1 Retarget  2025-01-05     NaN       404300   

   clicks     ctr  purchases  revenue   roas creative_type  \
0  4313.0  0.0183         80  1514.28   2.37         Image   
1  5429.0  0.0197         94  4152.81  11.11         Video   
2  9830.0  0.0211        240  4893.43   6.95           UGC   
3  2424.0  0.0125         61  1330.10   3.01         Image   
4  6372.0  0.0158        181  7027.20  51.52         Video   

                                    creative_message audience_type   platform  \
0  Breathable organic cotton th

In [7]:
print("--- Actual Column Names ---")
print(df.columns.tolist())

--- Actual Column Names ---
['campaign_name', 'adset_name', 'date', 'spend', 'impressions', 'clicks', 'ctr', 'purchases', 'revenue', 'roas', 'creative_type', 'creative_message', 'audience_type', 'platform', 'country']


In [9]:
df['date'] = pd.to_datetime(df['date'])

data_summary = {
    'Total Spend': df['spend'].sum(),
    'Total Revenue': df['revenue'].sum(),
    'Overall ROAS': df['revenue'].sum() / df['spend'].sum(),
    'Average CTR': df['ctr'].mean(),
    'Start Date': df['date'].min().strftime('%Y-%m-%d'),
    'End Date': df['date'].max().strftime('%Y-%m-%d'),
    'Total Days': df['date'].nunique()
}

summary_df = pd.DataFrame(data_summary, index=[0])

print("\n--- Data Agent Summary Output ---")
print(summary_df.T) # Transpose to make it easier to read


--- Data Agent Summary Output ---
                         0
Total Spend      2105579.9
Total Revenue  12265700.72
Overall ROAS      5.825331
Average CTR       0.013066
Start Date      2025-01-01
End Date        2025-03-31
Total Days              90


In [11]:
df.columns = df.columns.str.lower().str.strip()
df['date'] = pd.to_datetime(df['date'])


daily_df = df.groupby('date').agg(
    # Sum totals
    spend=('spend', 'sum'),
    revenue=('revenue', 'sum'),
    impressions=('impressions', 'sum'),
    clicks=('clicks', 'sum'),
    # Count unique creative/campaign activity
    campaign_count=('campaign_name', 'nunique')
).reset_index()

daily_df['daily_roas'] = daily_df['revenue'] / daily_df['spend']

daily_df['daily_ctr'] = (daily_df['clicks'] / daily_df['impressions']) * 100 

print("--- Daily Aggregated Data (First 5 Days) ---")
print(daily_df.head())

--- Daily Aggregated Data (First 5 Days) ---
        date     spend    revenue  impressions    clicks  campaign_count  \
0 2025-01-01  21427.75  112993.75     11411336  153799.0              36   
1 2025-01-02  24977.68  158374.54     14673489  198268.0              40   
2 2025-01-03  24362.07  131669.13     12948791  176331.0              34   
3 2025-01-04  23681.91  160393.11     14654372  192174.0              37   
4 2025-01-05  21452.16  112283.65     10413712  128574.0              31   

   daily_roas  daily_ctr  
0    5.273244   1.347774  
1    6.340643   1.351199  
2    5.404677   1.361756  
3    6.772811   1.311377  
4    5.234142   1.234661  


In [13]:
latest_date = daily_df['date'].max()

start_date_7d = latest_date - pd.Timedelta(days=6)

last_7d_df = daily_df[daily_df['date'] >= start_date_7d].copy()

data_agent_summary = {
    'Timeframe_Start': start_date_7d.strftime('%Y-%m-%d'),
    'Timeframe_End': latest_date.strftime('%Y-%m-%d'),
    'Daily_Performance': last_7d_df[['date', 'daily_roas', 'daily_ctr', 'spend']].to_dict('records')
}

print("\n--- Last 7 Days Daily Performance Summary (for Insight Agent) ---")
print(last_7d_df[['date', 'daily_roas', 'daily_ctr', 'spend']].set_index('date'))


--- Last 7 Days Daily Performance Summary (for Insight Agent) ---
            daily_roas  daily_ctr     spend
date                                       
2025-03-25    5.837421   1.246753  22262.75
2025-03-26    5.309130   1.261734  24592.22
2025-03-27    5.470381   1.346575  22907.85
2025-03-28    5.929069   1.262730  20926.73
2025-03-29    5.534783   1.163853  22646.14
2025-03-30    4.448009   1.237058  26046.53
2025-03-31    4.725396   1.232229  24644.90
